In [69]:
import pandas as pd
import numpy as np
from datetime import datetime
denver_path = "./denvercrime.csv"
crime_code_path = "./offense_codes.csv"
holidayDataDenver = pd.read_csv('Denver_Holiday.csv')
holidayDataVancouver = pd.read_csv('Vancouver_Holiday.csv')
eventDataDenver = pd.read_csv('Denver_events.csv')
df = pd.read_csv(denver_path)[:10]
crime_code_df = pd.read_csv(crime_code_path)

In [70]:
crime_code_index = []
crime_details=[]
for index, row in crime_code_df.iterrows():
    crime_code_index.append((row["OFFENSE_CODE"],row["OFFENSE_CODE_EXTENSION"]))
    crime_details.append(row["OFFENSE_TYPE_NAME"])

In [71]:
def time_cov(raw_in):
    if type(raw_in)==float:
        return raw_in,raw_in,raw_in,raw_in,raw_in,raw_in
    datetime_raw = datetime.strptime(raw_in, '%m/%d/%Y %I:%M:%S %p')
    date = int(datetime_raw.strftime("%Y%m%d"))
    hour = datetime_raw.hour
    minute = datetime_raw.minute
    second = datetime_raw.second
    dow = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"][datetime_raw.weekday()]
    return date,hour,minute,second,dow,datetime_raw


In [72]:
fact_df = pd.DataFrame(columns=['location_Key','crime_key', 'date_key','event_key', 'is_traffic','is_fatal','is_nighttime'])
location_df = pd.DataFrame(columns=['location_Key','city','location_name', 'neighborhood','latitude','longitude', 'crime_rate'])
crime_df= pd.DataFrame(columns=['crime_key','crime_category_name', 'crime_type_name','crime_details',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second"])
#no need to create twice in Vancouver
date_df = pd.DataFrame(columns=['Date_Key', 'Year','Month','Day', 'Day_of_Week','Is_Weekend','Is_Holiday_Vancouver','Holiday_Name_Vancouver','Is_Holiday_Denver','Holiday_Name_Denver'])
event_df = pd.DataFrame(columns=['event_key','event_name','event_type','event_location','event_neighbourhood','event_city','event_start_date','event_end_date','event_location_size'])


In [73]:
#Full Date Dimesion
start='20150101'
end='20191231'
s = pd.Series(pd.date_range(start, end))
date_df["Date_Key"] = s.dt.strftime('%Y%m%d')
date_df["Day_of_Week"] = s.dt.weekday_name
date_df['Is_Weekend'] = ((pd.to_numeric( s.dt.weekday)) // 5 == 1).astype('int64')
date_df["Year"] = s.dt.year
date_df["Month"] = s.dt.month
date_df["Day"] = s.dt.day
date_df["Is_Holiday_Denver"] = 0
date_df["Is_Holiday_Vancouver"] = 0
#Put Denver Holiday csv into Date Dimension
for idx_hol, row_hol in holidayDataDenver.iterrows():
    for idx_date, row_date in date_df.iterrows():
        if row_hol['YEAR']==row_date['Year'] and row_hol['MONTH']==row_date['Month'] and row_hol['DAY']== row_date['Day']:
            date_df.at[idx_date, 'Is_Holiday_Denver'] = 1
            date_df.at[idx_date, 'Holiday_Name_Denver'] = row_hol['NAME']
#Put Vancouver Holiday csv into Date Dimension
for idx_hol, row_hol in holidayDataVancouver.iterrows():
    for idx_date, row_date in date_df.iterrows():
        if row_hol['YEAR']==row_date['Year'] and row_hol['MONTH']==row_date['Month'] and row_hol['DAY']== row_date['Day']:
            date_df.at[idx_date, 'Is_Holiday_Vancouver'] = 1
            date_df.at[idx_date, 'Holiday_Name_Vancouver'] = row_hol['NAME']
date_df.head()

,Date_Key,Year,Month,Day,Day_of_Week,Is_Holiday_Vancouver,Holiday_Name_Vancouver,Is_Holiday_Denver,Holiday_Name_Denver,Is_Weekend
0,20150101,2015,1,1,Thursday,1,New Year's Day,1,New Year’s Day,0
1,20150102,2015,1,2,Friday,0,NaN,0,NaN,0
2,20150103,2015,1,3,Saturday,0,NaN,0,NaN,1
3,20150104,2015,1,4,Sunday,0,NaN,0,NaN,1
4,20150105,2015,1,5,Monday,0,NaN,0,NaN,0


In [74]:
#Event Dimension only has Denver Event
for idx_event, row_event in eventDataDenver.iterrows():
    event_row=[idx_event+3000000]
    event_row += [row_event['NAME'],row_event['TYPE'],row_event['LOCATION'],row_event['NEIGHBOURHOOD'],'Denver',row_event['DATE_START'],row_event['DATE_END'],row_event['LOCATION-SIZE']]
    event_df.loc[len(event_df)] = event_row
event_df.head()

,event_key,event_name,event_type,event_location,event_neighbourhood,event_city,event_start_date,event_end_date,event_location_size
0,3000000,test123,music,1601 Arapahoe St,montbello,Denver,20160613,20160615,address
1,3000001,test212,music,NaN,NaN,Denver,20160304,20160304,city


In [77]:
for index, row in df.iterrows():
    s_d,s_h,s_m,s_s,s_dow,s_dt = time_cov(row["FIRST_OCCURRENCE_DATE"])
    if not(2015<=s_dt.year<=2019):
        continue
    location_id = index + 1000000
    city = "Denver"
    location = row["INCIDENT_ADDRESS"]
    neighborhood = row["NEIGHBORHOOD_ID"]
    lat = row["GEO_LAT"]
    lon = row["GEO_LON"]
    crime_rate = 0
    location_df.loc[index] = [location_id,city,location,neighborhood,lat,lon,crime_rate]
    
    
    crime_id = index + 2000000
    crime_category_name = row["OFFENSE_CATEGORY_ID"]
    crime_type_name = row["OFFENSE_TYPE_ID"]
    crime_detail = crime_details[crime_code_index.index((row["OFFENSE_CODE"],row["OFFENSE_CODE_EXTENSION"]))]
    
    e_d,e_h,e_m,e_s,e_dow,e_dt = time_cov(row["LAST_OCCURRENCE_DATE"])
    r_d,r_h,r_m,r_s,r_dow,r_dt = time_cov(row["REPORTED_DATE"])
    crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,crime_detail,s_h,s_m,s_s,e_h,e_m,e_s,r_h,r_m,r_s]
    
    
    date_id = int(s_dt.strftime("%Y%m%d"))
    
    
    #Event
    event_id = np.nan
    for idx_event, row_event in event_df.iterrows():
        #the crime is in the event period
        if date_id>=row_event['event_start_date'] and date_id<=row_event['event_end_date']:
            #the event is city size and there is no event assign to the crime
            if row_event['event_location_size'] == 'city':
                if np.isnan(event_id):
                    event_id = row_event['event_key']
            #the event is not city size
            else:
                #the neighborhood are same
                if neighborhood == row_event['event_neighbourhood']:
                    event_id = row_event['event_key']
                    
    
    
    is_night = 0 if 7 <= s_h <= 18 else 1
    
    fatal_keywords=['FATAL','DEAD','DEATH','KILL','HOMICIDE','MURDER']
    is_fatal = 0
    for keyword in fatal_keywords:
        for name in [crime_category_name,crime_type_name,crime_detail]:
            if keyword in name.upper():
                is_fatal = 1

    
    fact_df.loc[index] = [location_id,crime_id,date_id,event_id,row["IS_TRAFFIC"],is_fatal,is_night]

In [78]:
location_df.head()

,location_Key,city,location_name,neighborhood,latitude,longitude,crime_rate
0,1000000,Denver,NaN,montbello,39.773188,-104.809881,0
1,1000001,Denver,NaN,gateway-green-valley-ranch,39.785649,-104.781434,0
2,1000002,Denver,2932 S JOSEPHINE ST,wellshire,39.663490,-104.957381,0
3,1000003,Denver,705 S COLORADO BLVD,belcaro,39.702698,-104.941440,0
4,1000004,Denver,2810 E 1ST AVE,cherry-creek,39.717107,-104.955370,0


In [79]:
crime_df.head()

,crime_key,crime_category_name,crime_type_name,crime_details,start_hour,start_minute,start_second,end_hour,end_minute,end_second,report_hour,report_minute,report_second
0,2000000,all-other-crimes,weapon-unlawful-discharge-of,Unlawful discharge of a weapon,23,31,0,NaN,NaN,NaN,23,31,0
1,2000001,larceny,theft-other,Theft - other,12,30,0,16,55,0,17,53,0
2,2000002,theft-from-motor-vehicle,theft-items-from-vehicle,Theft of items from a vehicle,20,0,0,8,0,0,21,2,0
3,2000003,larceny,theft-other,Theft - other,19,20,0,NaN,NaN,NaN,22,29,0
4,2000004,larceny,theft-shoplift,Shoplifting,20,53,0,NaN,NaN,NaN,16,9,0


In [80]:
# fact_df convert float to int
for col in fact_df.columns:
    if col !="event_key":
        fact_df[[col]]=fact_df[[col]].astype(int)

In [81]:
fact_df.head()

,location_Key,crime_key,date_key,event_key,is_traffic,is_fatal,is_nighttime
0,1000000,2000000,20160615,3000000,0,0,1
1,1000001,2000001,20171011,NaN,0,0,0
2,1000002,2000002,20160304,3000001,0,0,1
3,1000003,2000003,20180130,NaN,0,0,1
4,1000004,2000004,20170622,NaN,0,0,1


In [11]:
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0
